In [29]:
import torch

# read text file
def read_file(filename):
    with open(filename) as f:
        lines = f.readlines()
        lines = [x.strip() for x in lines]
    return lines

Clinical Labeling


In [30]:
import pandas as pd

df = pd.read_excel('/home/yhchoi/FSTrain_0205/src/labels/BrainMRI_PatientData.xlsx')
df.drop_duplicates(subset='Filename', keep='first', inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# reindex
df.reset_index(drop=True, inplace=True)
df['labels'] = 0
df.loc[df['pathogen'] == 'Bacterial', 'labels'] = 1
df.loc[df['pathogen'] == 'Tuberculosis', 'labels'] = 2
df.loc[df['pathogen'] == 'Viral', 'labels'] = 3
df

,Filename,ID,Subdir_YMD,Subdir_HMS,pathogen,mRS_at_discharge,age,sex,labels
0,10291451_10291451_1001_3d_t1_tfe_sag_f_gd,10291451,2021-04-01,22-22-36,Autoimmune,3,26,2,0
1,10335105_1001_3d_t1_tfe_sag_f_gd,10335105,2021-05-01,22-31-29,Autoimmune,2,59,1,0
2,10335105_901_3d_t1_bb_imsde_sag_f_gdcompo,10335105,2021-05-01,22-31-29,Autoimmune,2,59,1,0
3,10335105_2001_3d_t1_tfe_sag_f_gd,10335105,2021-05-01,22-28-52,Autoimmune,2,59,1,0
4,10335105_901_3d_t1_tfe_sag_f_gd,10335105,2021-06-01,22-35-53,Autoimmune,2,59,1,0
...,...,...,...,...,...,...,...,...,...
445,9908140_9908140_801_3d_t1_tfe_sag_f_gd,9908140,2020-01-01,02-55-17,Viral,1,52,2,3
446,9915963_1301_3d_t1_tfe_sag_f_gd,9915963,2020-02-01,02-58-13,Viral,0,39,1,3
447,9929551_1201_3d_t1_tfe_sag_f_gd,9929551,2020-02-01,03-01-06,Viral,0,20,2,3
448,9940380_1601_3d_t1_tfe_sag_f_gd,9940380,2020-03-01,03-04-46,Viral,0,22,2,3


In [31]:
df_ext = pd.read_excel('/home/yhchoi/FSTrain_0205/extdata_labels/external_validation_set_table (1).xlsx', sheet_name='Sheet1')
df_ext

,ID,True_ID,Admission,Discharge,LOS,Age,Sex,label,Pathogen,Mentality_abnormal,...,Ventilator,ICU,WBC,Hb,PLT,CRP,Procalcitonin,CSF_WBC,CSF_protein,CSF_glucose
0,10387855,10407969,44376,44392,16,35,1,3,EBV,1,...,1,1,7.55,14.80,288.0,90.40,0.09,17.0,68.2,59.0
1,10201440,10323715,44396,44410,14,68,2,3,NaN,1,...,0,0,8.48,14.00,181.0,10.00,0.21,22.0,128.8,83.0
2,10412107,10421373,44401,44447,46,19,2,0,ADEM,1,...,1,1,11.48,9.00,360.0,23.60,0.38,6.0,35.4,62.0
3,10415565,10422943,44402,44419,17,57,1,3,NaN,1,...,0,0,8.02,14.60,199.0,1.60,0.13,13.0,102.2,64.0
4,4811358,5047748,44410,44413,3,80,2,1,Listeria,1,...,0,0,17.20,9.50,115.0,226.65,7.49,230.0,361.7,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1491276,7081314,45207,45218,11,76,1,3,NaN,1,...,0,0,12.10,12.85,183.5,101.30,0.11,55.0,144.5,82.0
87,3710227,7697335,45254,45274,20,45,1,3,NaN,0,...,0,0,6.37,13.10,327.0,66.00,NaN,280.0,397.5,47.0
88,1782923,5991315,45276,45290,14,72,2,3,NaN,0,...,0,0,5.87,14.80,162.0,0.20,0.02,19.0,78.5,53.0
89,10112596,10276653,45277,45283,6,35,2,3,Entero,0,...,0,0,7.07,10.40,211.0,5.60,0.08,10.0,36.5,56.0


In [51]:
filenames = read_file('/home/yhchoi/24BrainMRI_Data/Data/text_files/nifti_caudate_left_padded_paths.txt')
filenames = [x.split('/')[-1] for x in filenames]
len(filenames)

624

In [52]:
ids = [filename.split('_')[2] for filename in filenames]
ids

['4459462',
 '9689569',
 '9725841',
 '9725841',
 '2079475',
 '9661322',
 '943738',
 '2298726',
 '9908140',
 '4459462',
 '1840003',
 '3711600',
 '9510429',
 '2393729',
 '3760709',
 '9788243',
 '1921943',
 '2598476',
 '4402543',
 '1886012',
 '2565592',
 '3639667',
 '3620258',
 '1197473',
 '2622879',
 '2588984',
 '4031557',
 '9661322',
 '4157697',
 '9689179',
 '3804316',
 '3922853',
 '9981226',
 '2536732',
 '2500950',
 '4773466',
 '9643603',
 '3651066',
 '4139232',
 '9529604',
 '3483199',
 '4545081',
 '3648041',
 '3648041',
 '4629993',
 '2271187',
 '3764993',
 '4353506',
 '3744094',
 '2813795',
 '2256984',
 '3685204',
 '4353506',
 '4654001',
 '1058600',
 '4738528',
 '9954319',
 '9689179',
 '4353506',
 '9665431',
 '9442409',
 '3651066',
 '4178419',
 '3575651',
 '4263734',
 '3458871',
 '2895327',
 '3764993',
 '4647342',
 '3257089',
 '3151976',
 '2729485',
 '4353506',
 '1649405',
 '2419950',
 '4082051',
 '3035868',
 '3461660',
 '2343040',
 '3075866',
 '9733447',
 '9878731',
 '4401359',
 '427

In [62]:
filenames_internal = filenames[604:]
filenames_internal = [filename.replace('.nii.gz', '').replace('_caudate_left', '') for filename in filenames_internal]
filenames_internal

['external_217_2977453_2977453_1201_3d_t1_tfe_sag_f_gd',
 'external_218_2085068_2085068_1301_sag_fspgr_bravo_gd',
 'external_219_2940531_2940531_1201_3d_t1_tfe_sag_f_gd',
 'external_220_10291451_10291451_1001_3d_t1_tfe_sag_f_gd',
 'external_221_3151976_3151976_801_3d_t1_tfe_sag_f_gd',
 'external_222_2781450_1001_3d_t1_tfe_sag_f_gd',
 'external_223_2715758_1201_3d_t1_tfe_sag_f_gd',
 'external_224_10197322_1301_3d_t1_tfe_sag_f_gd',
 'external_226_10335105_901_3d_t1_tfe_sag_f_gd',
 'external_228_9624039_901_3d_t1_tfe_sag_f_gd',
 'external_229_3257089_301_3d_sag',
 'external_230_10335105_901_3d_t1_tfe_sag_f_gd',
 'external_231_10381560_1101_3d_t1_tfe_sag_f_gd',
 'external_232_9824440_801_3d_t1_tfe_sag_f_gd',
 'external_233_10379982_801_3d_t1_tfe_sag_f_gd',
 'external_234_10335105_2001_3d_t1_tfe_sag_f_gd',
 'external_235_2715758_1401_3d_t1_tfe_sag_f_gd',
 'external_236_4782566_801_3d_t1_tfe_sag_f_gd',
 'external_237_10335105_901_3d_t1_bb_imsde_sag_f_gdcompo',
 'external_238_10335105_1001_3d

In [63]:
ids_internal = [filename.split('_')[2] for filename in filenames_internal]
len(ids_internal)

20

In [64]:
import pandas as pd

df = pd.read_excel('/home/yhchoi/FSTrain_0205/src/labels/BrainMRI_PatientData.xlsx')
df.drop_duplicates(subset='Filename', keep='first', inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# reindex
df.reset_index(drop=True, inplace=True)
df['labels'] = 0
df.loc[df['pathogen'] == 'Bacterial', 'labels'] = 1
df.loc[df['pathogen'] == 'Tuberculosis', 'labels'] = 2
df.loc[df['pathogen'] == 'Viral', 'labels'] = 3
df

,Filename,ID,Subdir_YMD,Subdir_HMS,pathogen,mRS_at_discharge,age,sex,labels
0,10291451_10291451_1001_3d_t1_tfe_sag_f_gd,10291451,2021-04-01,22-22-36,Autoimmune,3,26,2,0
1,10335105_1001_3d_t1_tfe_sag_f_gd,10335105,2021-05-01,22-31-29,Autoimmune,2,59,1,0
2,10335105_901_3d_t1_bb_imsde_sag_f_gdcompo,10335105,2021-05-01,22-31-29,Autoimmune,2,59,1,0
3,10335105_2001_3d_t1_tfe_sag_f_gd,10335105,2021-05-01,22-28-52,Autoimmune,2,59,1,0
4,10335105_901_3d_t1_tfe_sag_f_gd,10335105,2021-06-01,22-35-53,Autoimmune,2,59,1,0
...,...,...,...,...,...,...,...,...,...
445,9908140_9908140_801_3d_t1_tfe_sag_f_gd,9908140,2020-01-01,02-55-17,Viral,1,52,2,3
446,9915963_1301_3d_t1_tfe_sag_f_gd,9915963,2020-02-01,02-58-13,Viral,0,39,1,3
447,9929551_1201_3d_t1_tfe_sag_f_gd,9929551,2020-02-01,03-01-06,Viral,0,20,2,3
448,9940380_1601_3d_t1_tfe_sag_f_gd,9940380,2020-03-01,03-04-46,Viral,0,22,2,3


In [65]:
label_dict = dict(zip(df['ID'], df['labels']))
prognosis_dict = dict(zip(df['ID'], df['mRS_at_discharge']))
age_dict = dict(zip(df['ID'], df['age']))

In [66]:
internal_labels = [label_dict[int(x)] for x in ids_internal]
internal_prognosis = [prognosis_dict[int(x)] for x in ids_internal]
internal_age = [age_dict[int(x)] for x in ids_internal]

In [67]:
len(internal_age)

20

In [68]:
filenames_external = filenames[413:604]
filenames_external = [filename.replace('.nii.gz', '').replace('_caudate_left', '') for filename in filenames_external]
filenames_external

['external_1_2736962_20230701180512_801_3d_t1_tfe_sag_f_gd',
 'external_2_10916847_20230901030716_1301_sag_fspgr_bravo_gd',
 'external_3_11191153_20230601095139_201_3d_sag',
 'external_4_2478866_20220401155930_801_3d_t1_tfe_sag_f_gd',
 'external_5_10830481_20230801024210_1001_3d_t1_mdixon_sag_fs_gd',
 'external_7_10412107_20210801143116_901_3d_t1_tfe_sag_f_gd',
 'external_8_10456078_20211001170608_601_3d_sag',
 'external_9_10804449_20230501022257_1301_sag_fspgr_bravo_gd',
 'external_10_487703_20221101230141_1401_3d_t1_tfe_sag_f_gd',
 'external_11_4868223_20221101011652_1201_3d_t1_tfe_sag_f_gd',
 'external_12_3257089_20220701202527_901_3d_t1_tfe_sag_f_gd',
 'external_13_10705995_20220501233726_201_3d_t1_sag',
 'external_15_10444943_20210901161107_201_3d_sag',
 'external_16_10444943_20210901160316_201_3d_sag',
 'external_17_494693_20221101215632_1301_3d_t1_tfe_sag_f_gd',
 'external_19_10456078_20210901171258_401_3d_sag',
 'external_22_10629335_20230201214817_701_3d_t1_tfe_sag_f_gd',
 'ex

In [69]:
filenames_external_true = [filename for filename in filenames_external if int(filename.split('_')[2]) in df_ext['ID'].values]
ids_external_true = [int(filename.split('_')[2]) for filename in filenames_external_true]
len(ids_external_true)

191

In [70]:
filenames_external_false = ['/home/yhchoi/24BrainMRI_Data/Data/extractions/INSULA_CROPPED/right/' + filename for filename in filenames_external if int(filename.split('_')[2]) not in df_ext['ID'].values]
for i in filenames_external_false:
    print(i)
len(filenames_external_false)

0

In [71]:
label_dict_ext = dict(zip(df_ext['ID'], df_ext['label']))
prognosis_dict_ext = dict(zip(df_ext['ID'], df_ext['mRS_at_discharge']))
age_dict_ext = dict(zip(df_ext['ID'], df_ext['Age']))

In [73]:
external_labels = [label_dict_ext[int(x)] for x in ids_external_true]
external_prognosis = [prognosis_dict_ext[int(x)] for x in ids_external_true]
external_age = [age_dict_ext[int(x)] for x in ids_external_true]
len(external_age)

191

In [75]:
all_labels = external_labels + internal_labels
all_prognosis = external_prognosis + internal_prognosis
all_age = external_age + internal_age
all_filenames = filenames_external_true + filenames_internal

all_filenames

['external_1_2736962_20230701180512_801_3d_t1_tfe_sag_f_gd',
 'external_2_10916847_20230901030716_1301_sag_fspgr_bravo_gd',
 'external_3_11191153_20230601095139_201_3d_sag',
 'external_4_2478866_20220401155930_801_3d_t1_tfe_sag_f_gd',
 'external_5_10830481_20230801024210_1001_3d_t1_mdixon_sag_fs_gd',
 'external_7_10412107_20210801143116_901_3d_t1_tfe_sag_f_gd',
 'external_8_10456078_20211001170608_601_3d_sag',
 'external_9_10804449_20230501022257_1301_sag_fspgr_bravo_gd',
 'external_10_487703_20221101230141_1401_3d_t1_tfe_sag_f_gd',
 'external_11_4868223_20221101011652_1201_3d_t1_tfe_sag_f_gd',
 'external_12_3257089_20220701202527_901_3d_t1_tfe_sag_f_gd',
 'external_13_10705995_20220501233726_201_3d_t1_sag',
 'external_15_10444943_20210901161107_201_3d_sag',
 'external_16_10444943_20210901160316_201_3d_sag',
 'external_17_494693_20221101215632_1301_3d_t1_tfe_sag_f_gd',
 'external_19_10456078_20210901171258_401_3d_sag',
 'external_22_10629335_20230201214817_701_3d_t1_tfe_sag_f_gd',
 'ex

In [76]:
import torch

torch.save(torch.tensor(all_labels), 'pathogen_labels_external.pt')
torch.save(torch.tensor(all_prognosis), 'prognosis_labels_external.pt')
torch.save(torch.tensor(all_age), 'age_labels_external.pt')
torch.save(all_filenames, 'filename_labels_external.pt')

In [79]:
# retrieve the indices of pathogen == 3 from all_labels
for pathogen in [0, 1, 2, 3]:
    indices = [i for i, x in enumerate(all_labels) if x == pathogen]
    print(len(indices))
    torch.save(torch.tensor(indices), f'pathogen_indices_{pathogen}_external.pt')

78
15
5
113


In [78]:
for pathogen in [0, 1, 2, 3]:
    indices = [i for i, x in enumerate(all_labels) if x == pathogen]
    ids = [all_filenames[i].split('_')[2] for i in indices]
    print(len(set(ids)))

29
8
3
66
